In [1]:
#clear all
%reset -f

#import packages
import numpy as np
import sys
import os
import pandas as pd
import mne
import matplotlib
import h5py
from sklearn.utils import resample
from mne_icalabel import label_components

root = 'F:/Documents/Science/MirRevAdaptEEG'
participants = list(range(0,32))
#specify which erp we are analyzing
erps = 'frn'

#pop up plots as separate window & interactive
%matplotlib qt
matplotlib.pyplot.close('all')

In [2]:
#setting up path/ directory
#access specific file for specific participant
def load_epochs(pp_num, root_dir, erp_path):

    root_directory = root_dir
    data_directory = os.path.join(root_directory, 'data/eeg/')
    id_directory = os.path.join(data_directory, 'p%03d/' % pp_num)
    pp_directory = os.path.join(id_directory, erp_path)
    filename = os.path.join(pp_directory, 'mra_p%03d_run0-epo.fif' % pp_num)

    epochs = mne.read_epochs(filename)

    return epochs, pp_directory

In [3]:
#get index of trials from behavioral data
#rdm before rot and rdm before mir are read in as two different behavioral data
def epoch_index(task, pp):
    if task == 'rot':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\ROT_learningcurve_degrees.csv')
    elif task == 'rdmrot':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\RDMROT_learningcurve_degrees.csv')
    elif task == 'rdmmir':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\RDMMIR_learningcurve_degrees.csv')
    elif task == 'aln':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\ALIGNED_learningcurve_degrees.csv')
    elif task == 'mir':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\MIR_learningcurve_degrees_ALL.csv')
   
    ppdata = list(df.iloc[:,pp+1])
    
    index = [i for i,v in enumerate(ppdata)]
    
    #epochs_out = epochs[index]
    numtrials = len(index)
    
    #return epochs_out, numtrials
    return index, numtrials

In [4]:
#separate function to get indexed epochs for random, since we have them before rotation or mirror
def trial_list_rdm(task, pp, index):
    df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\eeg\p%03d\%s\mra_p%03d_tasktrigindex.csv' % (pp, erps, pp))
    
    #even pp numbers would experience rotation first (need random before this)
    #need to also take into account missing triggers from previous tasks
    if pp % 2 == 0:
        if task == 'rdmrot':
            #get trial after first task (trial 48 to 95, 234 to 281)
            tasksdf = df[0:96]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16150]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)
                    
        elif task == 'rdmmir':
            #get trial after first task (trial 48 to 95)
            tasksdf = df[0:282]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16153]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)
            
    elif pp % 2 == 1:   
        if task == 'rdmrot':
            #get trial after first five tasks (trial 234 to 281)
            tasksdf = df[0:282]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16153]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)
            
        elif task == 'rdmmir':
            #get trial after first task (trial 48 to 95)
            tasksdf = df[0:96]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16150]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

    
    return(triallist)
    

In [5]:
#get matching trials from eeg data, where we account for missing triggers
def epoch_trials(task, pp, epochs, index):

    df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\eeg\p%03d\%s\mra_p%03d_tasktrigindex.csv' % (pp, erps, pp))
    
    if task == 'aln':
        #get trial numbers where tasktrig is 16149 and trigidx is 1
        tasksdf = df[0:48]
        subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16149]
        triallist = []
        for i in index:
            ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
            nint = int(ndat['trigidx'])
            if nint == 1:
                trial = int(ndat['ctrial'])
                triallist.append(trial)

        epochs = epochs[triallist]
    
    #for the other tasks, the order depends on their pp number
    #even pp numbers would experience rotation first (need random before this)
    #need to also take into account missing triggers from previous tasks
    if pp % 2 == 0:
        if task == 'rot':
            #rot trials are after first two tasks (trial 96 to 185)
            tasksdf = df[0:186]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16151]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]
            
        elif task == 'mir':
            #get trial after first task (trial 282 to 371)
            tasksdf = df[0:372]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16154]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]
            
    elif pp % 2 == 1:
        if task == 'rot':
            #get trial after first five tasks (trial 282 to 371)
            tasksdf = df[0:372]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16151]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]
            
        elif task == 'mir':
            #get trial after first five tasks (trial 96 to 185)
            tasksdf = df[0:186]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16154]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]
    
    return(epochs)

In [6]:
#save epochs
def output_condition_epochs(pp_num, data, ppdir, task):
    # Save the cleaned data :)
    out_fname = os.path.join(ppdir, 'p%03d_%s_tfr-epo.fif' % (pp_num, task))
    data.save(out_fname, overwrite = True)

In [7]:
#save epoched files for each trial type (aligned, random, rotation, mirror)
#each trial type can have separate loops, as these will be split further into sub-conditions later on (e.g. error magnitude)

# ALIGNED
for pp in participants:
    data, ppdir = load_epochs(pp, root_dir = root, erp_path = erps)
    idx, numtrials = epoch_index('aln', pp)
    aldata = epoch_trials('aln', pp, data, idx)
    #print(pp, numtrials)
    output_condition_epochs(pp, aldata, ppdir, 'aligned')

# ROTATION
for pp in participants:
    data, ppdir = load_epochs(pp, root_dir = root, erp_path = erps)
    idx, numtrials = epoch_index('rot', pp)
    pdata = epoch_trials('rot', pp, data, idx)
    output_condition_epochs(pp, pdata, ppdir, 'rot')

# RANDOM (need to combine rdmrot and rdmmir into one)
for pp in participants:
    
    data, ppdir = load_epochs(pp, root_dir = root, erp_path = erps)
    #rdmrot
    idx, numtrials = epoch_index('rdmrot', pp)
    rdmrot_data = trial_list_rdm('rdmrot', pp, idx)
    
    #rdmmir
    idx, numtrials = epoch_index('rdmmir', pp)
    rdmmir_data = trial_list_rdm('rdmmir', pp, idx)
    
    #combine both rdmrot and rdmmir
    rdmdat = rdmrot_data + rdmmir_data
    rdmdat.sort()
    
    pdata = data[rdmdat]
    
    #save as one file of epochs called rdm
    output_condition_epochs(pp, pdata, ppdir, 'rdm')
        
# MIRROR
for pp in participants:
    data, ppdir = load_epochs(pp, root_dir = root, erp_path = erps)
    idx, numtrials = epoch_index('mir', pp)
    pdata = epoch_trials('mir', pp, data, idx)
    output_condition_epochs(pp, pdata, ppdir, 'mir')


Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\frn\mra_p000_run0-epo.fif ...
    Found the data of interest:
        t =   -1500.00 ...    1495.00 ms
        0 CTF compensation matrices available
Not setting metadata
420 matching events found
No baseline correction applied
0 projection items activated
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p001\frn\mra_p001_run0-epo.fif ...
    Found the data of interest:
        t =   -1500.00 ...    1495.00 ms
        0 CTF compensation matrices available
Not setting metadata
420 matching events found
No baseline correction applied
0 projection items activated
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p002\frn\mra_p002_run0-epo.fif ...
    Found the data of interest:
        t =   -1500.00 ...    1495.00 ms
        0 CTF compensation matrices available
Not setting metadata
420 matching events found
No baseline correction applied
0 projection items activated
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p003\

    Found the data of interest:
        t =   -1500.00 ...    1495.00 ms
        0 CTF compensation matrices available
Not setting metadata
419 matching events found
No baseline correction applied
0 projection items activated
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p027\frn\mra_p027_run0-epo.fif ...
    Found the data of interest:
        t =   -1500.00 ...    1495.00 ms
        0 CTF compensation matrices available
Not setting metadata
420 matching events found
No baseline correction applied
0 projection items activated
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p028\frn\mra_p028_run0-epo.fif ...
    Found the data of interest:
        t =   -1500.00 ...    1495.00 ms
        0 CTF compensation matrices available
Not setting metadata
420 matching events found
No baseline correction applied
0 projection items activated
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p029\frn\mra_p029_run0-epo.fif ...
    Found the data of interest:
        t =   -1500.00 ...

        t =   -1500.00 ...    1495.00 ms
        0 CTF compensation matrices available
Not setting metadata
420 matching events found
No baseline correction applied
0 projection items activated
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p021\frn\mra_p021_run0-epo.fif ...
    Found the data of interest:
        t =   -1500.00 ...    1495.00 ms
        0 CTF compensation matrices available
Not setting metadata
420 matching events found
No baseline correction applied
0 projection items activated
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p022\frn\mra_p022_run0-epo.fif ...
    Found the data of interest:
        t =   -1500.00 ...    1495.00 ms
        0 CTF compensation matrices available
Not setting metadata
420 matching events found
No baseline correction applied
0 projection items activated
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p023\frn\mra_p023_run0-epo.fif ...
    Found the data of interest:
        t =   -1500.00 ...    1495.00 ms
        0 CTF com

        0 CTF compensation matrices available
Not setting metadata
420 matching events found
No baseline correction applied
0 projection items activated
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p015\frn\mra_p015_run0-epo.fif ...
    Found the data of interest:
        t =   -1500.00 ...    1495.00 ms
        0 CTF compensation matrices available
Not setting metadata
420 matching events found
No baseline correction applied
0 projection items activated
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p016\frn\mra_p016_run0-epo.fif ...
    Found the data of interest:
        t =   -1500.00 ...    1495.00 ms
        0 CTF compensation matrices available
Not setting metadata
420 matching events found
No baseline correction applied
0 projection items activated
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p017\frn\mra_p017_run0-epo.fif ...
    Found the data of interest:
        t =   -1500.00 ...    1495.00 ms
        0 CTF compensation matrices available
Not setting 

Not setting metadata
420 matching events found
No baseline correction applied
0 projection items activated
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p009\frn\mra_p009_run0-epo.fif ...
    Found the data of interest:
        t =   -1500.00 ...    1495.00 ms
        0 CTF compensation matrices available
Not setting metadata
420 matching events found
No baseline correction applied
0 projection items activated
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p010\frn\mra_p010_run0-epo.fif ...
    Found the data of interest:
        t =   -1500.00 ...    1495.00 ms
        0 CTF compensation matrices available
Not setting metadata
420 matching events found
No baseline correction applied
0 projection items activated
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p011\frn\mra_p011_run0-epo.fif ...
    Found the data of interest:
        t =   -1500.00 ...    1495.00 ms
        0 CTF compensation matrices available
Not setting metadata
420 matching events found
No baseline

In [8]:
#setting up path/ directory
#access trial type epoched data for each participant
def load_tfr_epochs(pp_num, root_dir, erp_path, task):

    root_directory = root_dir
    data_directory = os.path.join(root_directory, 'data/eeg/')
    id_directory = os.path.join(data_directory, 'p%03d/' % pp_num)
    pp_directory = os.path.join(id_directory, erp_path)
    filename = os.path.join(pp_directory, 'p%03d_%s_tfr-epo.fif' % (pp_num, task))

    epochs = mne.read_epochs(filename)

    return epochs, pp_directory

In [9]:
#set up parameters for time-frequency morlet convolution
def set_tfr_params(l_freq = 6, h_freq = 35, num_freq = 50, cycle = 6, samp_rate = 200):
    freqs = np.logspace(*np.log10([l_freq, h_freq]), num = num_freq)
    n_cycles = cycle
    sfreq = samp_rate
    
    return freqs, n_cycles, sfreq

In [10]:
#save tfr'd data
def output_tfr_data(root_dir, data, task, output):
    # Save the tfr'd data
    root_directory = root_dir
    data_directory = os.path.join(root_directory, 'data/eeg/tfr_ana/%s' % (erps))
    out_fname = os.path.join(data_directory, '%s_%s-tfr.h5' % (task, output))
    mne.time_frequency.write_tfrs(out_fname, tfr=data, overwrite = True)

In [11]:
#morlet convolution for each participant, according to trial types - POWER

perturb_conds = ['aligned', 'rdm', 'rot', 'mir']
# perturb_conds = ['aligned']

for pcond in range(0, len(perturb_conds)):
    pp_power = []
    for pp in participants:
        data, ppdir = load_tfr_epochs(pp_num = pp, root_dir = root, erp_path = erps, task = perturb_conds[pcond])
        freqs, n_cycles, sfreq = set_tfr_params()
        
        power = mne.time_frequency.tfr_morlet(
            data,
            freqs=freqs,
            n_cycles=n_cycles,
            use_fft=False,
            decim=1,
            n_jobs=None,
            return_itc = False,
            average = False
        )
        #average epochs within participant first, then across participants
        win_power = power.average()
        pp_power.append(win_power)
        
    alldat = mne.grand_average(pp_power, interpolate_bads=True, drop_bads=True)
        
    
    if pcond == 0:
        output_tfr_data(root_dir = root, data = alldat, task = "aligned", output = "power")

    elif pcond == 1:
        output_tfr_data(root_dir = root, data = alldat, task = "rdm", output = "power")

    elif pcond == 2:
        output_tfr_data(root_dir = root, data = alldat, task = "rot", output = "power")

    elif pcond == 3:
        output_tfr_data(root_dir = root, data = alldat, task = "mir", output = "power")


Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\frn\p000_aligned_tfr-epo.fif ...
    Found the data of interest:
        t =   -1500.00 ...    1495.00 ms
        0 CTF compensation matrices available
Not setting metadata
48 matching events found
No baseline correction applied
0 projection items activated


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:   13.0s finished


Not setting metadata
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p001\frn\p001_aligned_tfr-epo.fif ...
    Found the data of interest:
        t =   -1500.00 ...    1495.00 ms
        0 CTF compensation matrices available
Not setting metadata
48 matching events found
No baseline correction applied
0 projection items activated


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:   13.0s finished


Not setting metadata
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p002\frn\p002_aligned_tfr-epo.fif ...
    Found the data of interest:
        t =   -1500.00 ...    1495.00 ms
        0 CTF compensation matrices available
Not setting metadata
48 matching events found
No baseline correction applied
0 projection items activated


...

Not setting metadata
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p030\frn\p030_mir_tfr-epo.fif ...
    Found the data of interest:
        t =   -1500.00 ...    1495.00 ms
        0 CTF compensation matrices available
Not setting metadata
90 matching events found
No baseline correction applied
0 projection items activated


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    1.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:   23.8s finished


Not setting metadata
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p031\frn\p031_mir_tfr-epo.fif ...
    Found the data of interest:
        t =   -1500.00 ...    1495.00 ms
        0 CTF compensation matrices available
Not setting metadata
90 matching events found
No baseline correction applied
0 projection items activated


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    1.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:   23.9s finished


Not setting metadata
Identifying common channels ...


In [12]:
#load tfr'd data
def load_tfr_data(root_dir, task, output):
    # Save the tfr'd data
    root_directory = root_dir
    data_directory = os.path.join(root_directory, 'data/eeg/tfr_ana/%s' % (erps))
    out_fname = os.path.join(data_directory, '%s_%s-tfr.h5' % (task, output))
    dat = mne.time_frequency.read_tfrs(out_fname)
    
    return dat

In [13]:
#plot average tfr for each of perturbation conditions/ trial types

perturb_conds = ['aligned', 'rdm', 'rot', 'mir']
#perturb_conds = ['aligned']

for pcond in range(0, len(perturb_conds)):
    data = load_tfr_data(root_dir=root, task = perturb_conds[pcond], output = "power")
    data[0].plot(baseline=(-1.3,-1.0), mode="logratio", combine='mean', tmin=-1.3, tmax=1.3, title="%s" % (perturb_conds[pcond]))

Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/tfr_ana/frn\aligned_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/tfr_ana/frn\rdm_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/tfr_ana/frn\rot_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/tfr_ana/frn\mir_power-tfr.h5 ...
Applying baseline correction (mode: logratio)


In [14]:
# #TOPO PLOTS of average tfr for each of perturbation conditions/ trial types

# perturb_conds = ['aligned', 'rdm', 'rot', 'mir']
# #perturb_conds = ['aligned']

# for pcond in range(0, len(perturb_conds)):
#     data = load_tfr_data(root_dir=root, task = perturb_conds[pcond], output = "power")
#     data[0].plot_topo(baseline=(-1.3,-1.0), mode="logratio", tmin=-1.3, tmax=1.3, title="%s" % (perturb_conds[pcond]))

In [15]:
# #morlet convolution for each participant, according to trial types - ITC

# perturb_conds = ['aligned', 'rdm', 'rot', 'mir']
# #perturb_conds = ['aligned']

# for pcond in range(0, len(perturb_conds)):
#     #pp_power = []
#     pp_itc = []
#     for pp in participants:
#         data, ppdir = load_tfr_epochs(pp_num = pp, root_dir = root, erp_path = erps, task = perturb_conds[pcond])
#         freqs, n_cycles, sfreq = set_tfr_params()
        
#         power, itc = mne.time_frequency.tfr_morlet(
#             data,
#             freqs=freqs,
#             n_cycles=n_cycles,
#             use_fft=False,
#             decim=1,
#             n_jobs=None,
#             return_itc = True
#         )
#         #pp_power.append(power)
#         pp_itc.append(itc)
    
#     if pcond == 0:
#         #output_tfr_data(root_dir = root, data = pp_power, output = 'aligned_power')
#         output_tfr_data(root_dir = root, data = pp_itc, output = 'aligned_itc')
#     elif pcond == 1:
#         #output_tfr_data(root_dir = root, data = rdm_power, output = 'rdm_power')
#         output_tfr_data(root_dir = root, data = pp_itc, output = 'rdm_itc')
#     elif pcond == 2:
#         #output_tfr_data(root_dir = root, data = rot_power, output = 'rot_power')
#         output_tfr_data(root_dir = root, data = pp_itc, output = 'rot_itc')
#     elif pcond == 3:
#         #output_tfr_data(root_dir = root, data = mir_power, output = 'mir_power')
#         output_tfr_data(root_dir = root, data = pp_itc, output = 'mir_itc')
        
    
                
            

In [16]:
# # ONLY FOR CHECKING DATA
# #morlet convolution for each participant, according to trial types - POWER
# #modified version of cell above, so that we can procude tfr plots for each participant


# # perturb_conds = ['aligned', 'rdm', 'rot', 'mir']
# # perturb_conds = ['aligned']
# perturb_conds = ['rot']

# for pcond in range(0, len(perturb_conds)):
#     pp_power = []
#     for pp in participants:
#         data, ppdir = load_tfr_epochs(pp_num = pp, root_dir = root, erp_path = erps, task = perturb_conds[pcond])
#         freqs, n_cycles, sfreq = set_tfr_params()
        
#         power = mne.time_frequency.tfr_morlet(
#             data,
#             freqs=freqs,
#             n_cycles=n_cycles,
#             use_fft=False,
#             decim=1,
#             n_jobs=None,
#             return_itc = False,
#             average = False
#         )
#         #average epochs within participant first, then across participants
#         win_power = power.average()
#         pp_power.append(win_power)
        

In [17]:
# # FOR CHECKING DATA
# # tfr plot for each participant, cobining all channels
# for i in range(0, len(pp_power)):
#     pp_power[i].plot(baseline=(-1.3,-1.0), mode="logratio",combine='mean', tmin=-1.3, tmax=1.3)